In [1]:
# Добавить несколько вариантов стратификации. Первый вариант это релизовать уравновешивание нескольких классов 
# по наименьшему количеству элементов относительно таргета.
# 

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv(r"D:\ML\spo_2\spo\data\main.csv")
df.head()

,actual_guid,age_after_half_yead_exam,gender,parallel,students_count,spo_students_count,spo_students_precent,school_id,school_absence_count,school_absence_count_anomaly,...,had_mk,absence_count,absence_count_nulls,absence_percent,quantity_of_services,pay_form_fact,knowledge_topic_unique,year_start_unique,services_nulls,y
0,673a9e20-254c-4c2d-b67b-cc6155bd9799_nsi3,15,1,10,828,0,0.000000,595,308262,0,...,0,367,0,0.342670,4,1,2,3,0,0
1,428df0d7-1c9d-4a56-981c-28080754db28_nsi3,15,0,9,359,1,0.278552,1074,27680,0,...,0,3,0,0.003501,2,1,1,1,0,0
2,c573a475-c9f8-4d37-b581-8141f6b7d090_nsi3,16,0,9,311,0,0.000000,10,111895,0,...,0,377,0,0.428897,6,1,4,3,0,0
3,aa72723c-572e-4909-84f0-41e01a73c2c9_nsi3,14,0,9,322,0,0.000000,802,109904,0,...,0,235,0,0.391667,2,1,2,1,0,0
4,a492e13d-5d75-4d26-b459-b20512d35f02_nsi3,15,1,9,471,0,0.000000,590,146828,0,...,0,319,0,0.318046,2,0,1,2,0,0


In [4]:
columns_name = ['gender','parallel']
target_name = 'y'

In [5]:
uniques = {}

for col in columns_name:
    uniques[col] = df[col].unique()

uniques[target_name] = df[target_name].unique()

uniques

{'gender': array([1, 0], dtype=int64),
 'parallel': array([10,  9, 11, 12], dtype=int64),
 'y': array([0, 1], dtype=int64)}

In [6]:
columns = columns_name.copy()
columns.append(target_name)
df_dict = df[columns].value_counts().to_dict()
df_dict

{(1, 9, 0): 37470,
 (0, 9, 0): 37024,
 (0, 10, 0): 29772,
 (1, 10, 0): 29756,
 (0, 11, 0): 28435,
 (1, 11, 0): 27177,
 (1, 9, 1): 8165,
 (0, 9, 1): 5853,
 (1, 11, 1): 2665,
 (0, 11, 1): 1948,
 (1, 10, 1): 313,
 (0, 10, 1): 280,
 (0, 12, 0): 9,
 (1, 12, 0): 9,
 (1, 12, 1): 4,
 (0, 12, 1): 2}

In [7]:
df_dict_sorted = dict(sorted(df_dict.items(), key = lambda x: x[0]))
temp_list = []
min_value = max(df_dict_sorted.values()) + 1
c = 1

for index in df_dict_sorted:

    temp_list.append(index)

    if df_dict_sorted[index] < min_value:
        min_value = df_dict_sorted[index]

    if c % 2 == 0:
        for i in temp_list:
            df_dict_sorted[i] = min_value
        temp_list = []
        min_value = max(df_dict_sorted.values()) + 1
        
    c += 1

In [15]:
def sample_part_on_index(df, columns_name, index, value, random_state = None):

    for i in range(len(columns_name)):
        df = df[df[columns_name[i]] == index[i]]

    if random_state is not None:
        return df.sample(value, random_state=random_state)
    else:
        return df.sample(value)

In [18]:
sampled_dataframe = pd.DataFrame(columns=df.columns)
df_temp = df.copy()

for index in df_dict_sorted.keys():
    
    sampled_dataframe = pd.concat([sampled_dataframe, sample_part_on_index(df_temp, columns, index = index, value = df_dict_sorted[index])])

In [ ]:
def equalize(df, target_name, columns_name = list | None, random_state = int | None):
    